In [ ]:
#### import global modules
import os
import sys
import pandas as pd
from pathlib import Path
from yaml import safe_load
import google.oauth2.credentials
from google.cloud import bigquery
import numpy as np
# Set global vars
pth_project = Path(os.getcwd().split('notebooks')[0])
pth_data = pth_project / 'data'
pth_queries = pth_project / 'core' / 'queries'
pth_creds = pth_project / 'conf' / 'local' / 'project_config.yaml'
sys.path.insert(0, str(pth_project))
d_project_config = safe_load(pth_creds.open())
# d_params = safe_load((pth_project / 'core' / 'parameters' / 'common.yaml').open())['data_extract']

# import local modules
from core.utils.gcp import connect_bq_services
# from core.etl.extract import extract_bq_data, extract_pr_codes, format_conv_df, filter_convs

# Connect to google services
bq_client = connect_bq_services(d_project_config['gcp-project-name'])
pd.options.display.max_rows = 100

In [ ]:
def extract_bq_data(bq_client, sql=None, pth_query=None):
    if sql is not None:
        df = bq_client.query(sql).to_dataframe()
    elif pth_query is not None:
        sql = pth_query.read_text()
        df = bq_client.query(sql).to_dataframe()
    else:
        raise ValueError('`sql` or `pth_query` should be set')  
    return df

In [ ]:
Query='''

select * from `cio-datahub-enterprise-pr-183a.src_adc.bq_customer_daily_logins`

'''

In [ ]:
Customer_daily_login= extract_bq_data(bq_client,sql=Query)

In [ ]:
Customer_daily_login.info()

In [ ]:
Customer_daily_login.head()

In [ ]:
Customer_daily_login['login_type_desc'].value_counts()

In [ ]:
Customer_Daily_login_pivot_table=pd.pivot_table(Customer_daily_login, values='login_count', index=['login_dt_utc','customer_id'], columns='login_type_desc',
                          aggfunc={'login_count': np.sum}).reset_index()

In [ ]:
Customer_Daily_login_pivot_table.head()

In [ ]:
Customer_Daily_login_pivot_table.columns=['login_date_utc','customer_id','Mobile','Voice_Control_Devices','Web']

In [ ]:
Customer_Daily_login_pivot_table['login_date_utc'].min()

In [ ]:
Customer_Daily_login_pivot_table['login_date_utc'].max()

In [ ]:
Customer_Daily_login_pivot_table.head()

In [ ]:
Customer_Daily_login_pivot_table=Customer_Daily_login_pivot_table.fillna(0)

In [ ]:
Customer_Daily_login_pivot_table.head()

In [ ]:
config= bigquery.job.LoadJobConfig()

# config._properties['timePartitioning'] = {'field': 'Month_Year'}
config.write_disposition = bigquery.WriteDisposition.WRITE_TRUNCATE


Table_BQ = 'ADC.Customer_Daily_login'

bq_table_instance= bq_client.load_table_from_dataframe(Customer_Daily_login_pivot_table, Table_BQ,job_config=config)